# E-commerce Marketing and Sales Analysis

### 1. Business Context and Problem 

The objective of this project is to leverage data-driven insights to enhance customer acquisition, retention, and optimize revenue streams for an e-commerce company.

The focus areas include analyzing transactions, marketing spends, discount strategies, customer behavior, and product performance across a complete calendar year.

### Problem Statement

Through exploratory analysis and visualization, we aim to answer key business questions such as:

- Identify months with highest and lowest customer acquisition.
- Analyze retention patterns and customer lifetime value.
- Evaluate coupon effectiveness and marketing ROI.
- Segment customers using RFM analysis to tailor strategies.
- Understand how taxes and delivery charges influence purchasing behaviors.

### 2. Data Description

The project utilizes multiple datasets covering the period 1st Jan 2019 to 31st Dec 2019:

- Online Sales: Transaction-level sales data including quantity, pricing, and product details.
- Customers Data: Customer demographic information such as gender, location, and tenure.
- Discount Coupons: Information on discount codes and associated percentages across months and categories.
- Marketing Spend: Daily spend data across offline and online marketing channels.
- Tax Amount: GST percentages applicable by product categories.

### 3. Library Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
from scipy.stats import ttest_ind, f_oneway

### 4. Data Loading

In [ ]:
sales = pd.read_csv('../resources/Online_Sales.csv')
customers = pd.read_excel('../resources/CustomersData.xlsx')
discounts = pd.read_csv('../resources/Discount_Coupon.csv')
marketing = pd.read_csv('../resources/Marketing_Spend.csv')
tax = pd.read_excel('../resources/Tax_amount.xlsx')

### 5. Data Overview and initial data transformation

In [ ]:
# Convert dates to datetime object
sales['Transaction_Date'] = pd.to_datetime(sales['Transaction_Date'])
marketing['Date'] = pd.to_datetime(marketing['Date'])

# Create new Month columns
sales['Transaction_Month'] = sales['Transaction_Date'].dt.to_period('M')
marketing['Marketing_Month'] = marketing['Date'].dt.to_period('M')

# lower-casing all column names
sales.columns = sales.columns.str.lower()
customers.columns = customers.columns.str.lower()
discounts.columns = discounts.columns.str.lower()
marketing.columns = marketing.columns.str.lower()
tax.columns = tax.columns.str.lower()

# if all discounts are for 2019:
discounts['month'] = pd.to_datetime('2019-' + discounts['month'], format='%Y-%b').dt.to_period('M')

for df in (sales, discounts, tax):
    df['product_category'] = df['product_category'].str.strip().str.lower()
    
sales.rename(columns={'customerid':'customer_id'}, inplace=True)
customers.rename(columns={'customerid':'customer_id'}, inplace=True)

sales['revenue'] = sales['quantity'] * sales['avg_price']
sales['total_amount'] = sales['revenue'] + sales['delivery_charges']


In [ ]:
# Customer integrity check
# unique customers in sales
sales_customers = set(sales['customer_id'].unique())

# unique customers in customer
customers_list = set(customers['customer_id'].unique())

missing_customers = sales_customers - customers_list
print(f"No of customers in sales not found in customers table : {len(missing_customers)}")

In [ ]:
# Saving cleaned data
sales.to_csv('../eda_outputs/sales_cleaned.csv', index=False)
customers.to_csv('../eda_outputs/customers_cleaned.csv', index=False)
discounts.to_csv('../eda_outputs/discounts_cleaned.csv', index=False)
marketing.to_csv('../eda_outputs/marketing_cleaned.csv', index=False)
tax.to_csv('../eda_outputs/tax_cleaned.csv', index=False)

## Question 1  
**Identify the months with the highest and lowest acquisition rates.**  
*What strategies could be implemented to address the fluctuations and ensure consistent growth throughout the year?*

In [ ]:
# Compute each customer's first purchase month
first_purchase = (
    sales
    .groupby('customer_id')['transaction_date']
    .min()
    .dt.to_period('M')
    .rename('first_month')
    .reset_index()
)

# Count new customers per month
monthly_acquisition = (
    first_purchase
    .groupby('first_month')
    .size()
    .sort_index()
)

# Identify best and worst
best_month  = monthly_acquisition.idxmax()
best_count  = monthly_acquisition.max()
worst_month = monthly_acquisition.idxmin()
worst_count = monthly_acquisition.min()

print(f"Highest acquisition: {best_month} → {best_count} new customers")
print(f"Lowest  acquisition: {worst_month} → {worst_count} new customers")

### Results  
**Highest acquisition month:** January, 2019 with 215 new customers  
**Lowest acquisition month:** November with 68 new customers  

## Question 2  
**Analyze the data to determine if certain months consistently show higher or lower acquisition rates.**  
*How can the company capitalize on high-performing months and improve performance during slower periods?*

### 2.1 Business Story & Context  
> New-customer acquisition often follows a seasonal pattern.  
> By quantifying which calendar months over- or under-perform, we can time campaigns, budget, and product launches to maximize ROI and smooth out revenue dips.

In [ ]:
first_purchase2 = (
    sales
    .groupby('customer_id')['transaction_date']
    .min()
    .reset_index()
)

# Extract month name and order Jan→Dec
first_purchase2['month'] = first_purchase2['transaction_date'].dt.month_name()
month_order = [
    "January","February","March","April","May","June",
    "July","August","September","October","November","December"
]

# Count and assemble seasonality table
monthly_acq_by_month = (
    first_purchase2['month']
    .value_counts()
    .reindex(month_order)
    .fillna(0)
    .astype(int)
)
avg = monthly_acq_by_month.mean()
deviation = (monthly_acq_by_month - avg) / avg * 100
seasonality = pd.DataFrame({
    'acquisitions': monthly_acq_by_month,
    'pct_vs_avg': deviation.round(1)
})
sales.to_csv('../eda_outputs/acquisation_seasonality.csv', index=False)

# Visualizations 
plt.figure(figsize=(12, 6))
plt.bar(seasonality.index, seasonality['acquisitions'])
plt.title("New-Customer Acquisitions by Calendar Month")
plt.xlabel("Month")
plt.ylabel("Number of New Customers")
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.tight_layout()
plt.savefig('../images/acquisation_seasonality.png')
plt.show()

## Question 3  
**Identify periods with the strongest and weakest retention rates.**  
*What strategies could be implemented to improve retention during weaker months?*

### 3.1. Business Story & Context

While acquisition brings in new customers, retention ensures long-term revenue and brand loyalty.
Identifying strong and weak retention periods helps the business understand when customers tend to come back—and when they don’t—so that lifecycle campaigns and engagement strategies can be optimized.

In [ ]:
# Join first_month to sales
sales_fp = sales.merge(first_purchase, on='customer_id')

sales_fp['month'] = sales_fp['transaction_month']

# customers who made purchases in the month after their first
retention_flags = (
    sales_fp[sales_fp['month'] > sales_fp['first_month']]
    .assign(next_month = sales_fp['first_month'] + 1)
    .query("month == next_month")
    .groupby('first_month')['customer_id']
    .nunique()
    .rename('retained')
)

# new customers in each month
cohort_sizes = first_purchase.groupby('first_month')['customer_id'].nunique().rename('acquired')

# Combine and calculate retention rate
retention = pd.concat([cohort_sizes, retention_flags], axis=1).fillna(0)
retention['retention_rate'] = (retention['retained'] / retention['acquired']).round(2)

retention.to_csv('../eda_outputs/next_month_retention.csv', index=False)


# Visualiazation
plt.figure(figsize=(10, 6))
plt.plot(retention.index.astype(str), retention['retention_rate'], marker='o')
plt.title("Next-Month Customer Retention Rate by Acquisition Month")
plt.xlabel("Cohort (Acquisition) Month")
plt.ylabel("Retention Rate")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig('../images/next_month_retention.png')
plt.show()

### 3.2. Results

Based on the **next-month retention** rates:

| Cohort (Acq Month) | Retention Rate |
|--------------------|----------------|
| **Highest**        |                |
| 2019-06            | 15 %           |
| 2019-07            | 14 %           |
| **Lowest**         |                |
| 2019-01            |  6 %           |
| 2019-02,10         |  7 %           |


## Question 4  
**Analyze customer behavior during high-retention months and suggest ways to replicate this success throughout the year.**

### 4.1. Business Story & Context  
> Cohorts with strong retention often exhibit particular behaviors—higher AOV, smart coupon usage or favorite categories—that drive repeat purchases.  
> By uncovering these patterns, we can design campaigns in slower months that mimic their success.

In [ ]:
# Identify high-retention cohorts (≥ average retention rate)

avg_ret = retention['retention_rate'].mean()
high_retention_months = retention[retention['retention_rate'] >= avg_ret].index.tolist()


# For each high-retention cohort, analyze next-month behavior
behavior_list = []
for cohort in high_retention_months:
    custs = first_purchase[first_purchase['first_month'] == cohort]['customer_id']
    next_mon = cohort + 1
    df_next = sales[(sales['customer_id'].isin(custs)) & 
                    (sales['transaction_month'] == next_mon)]
    behavior_list.append({
        'cohort': str(cohort),
        'avg_order_value': df_next['total_amount'].mean(),
        'coupon_rate': (df_next['coupon_status']=='Used').mean(),
        'top_categories': ', '.join(
            df_next['product_category'].value_counts().head(3).index.tolist()
        )
    })

behavior_df = pd.DataFrame(behavior_list).set_index('cohort')

behavior_df.to_csv('../eda_outputs/retention_drivers.csv', index=False)

# Visualizations 
# Avg Order Value for high-retention cohorts
plt.figure(figsize=(8, 4))
behavior_df['avg_order_value'].plot(kind='bar')
plt.title('Avg Order Value in Next Month for High-Retention Cohorts')
plt.ylabel('Avg Order Value')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('../images/avg_order_value.png')
plt.show()

# Coupon Usage Rate for high-retention cohorts
plt.figure(figsize=(8, 4))
behavior_df['coupon_rate'].plot(kind='bar')
plt.title('Coupon Usage Rate in Next Month for High-Retention Cohorts')
plt.ylabel('Coupon Usage Rate')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('../images/coupon_usage.png')
plt.show()

### 4.2 Results  

| Cohort   | Avg Order Value ($) | Coupon Usage Rate | Top Categories                 |
|----------|---------------------|-------------------|--------------------------------|
| 2019-03  | 119.91              | 31.7%             | apparel, nest-usa, office      |
| 2019-04  | 110.28              | 28.3%             | apparel, nest-usa, office      |
| 2019-05  |  83.24              | 33.3%             | nest-usa, apparel, office      |
| 2019-06  |  55.11              | 42.0%             | apparel, nest-usa, office      |
| 2019-07  |  68.58              | 35.1%             | apparel, nest-usa, office      |
| 2019-08  | 127.67              | 38.9%             | nest-usa, apparel, nest        |
| 2019-11  | 153.00              | 36.4%             | nest-usa, nest, office         |

---

## Question 5  
**Compare the revenue generated by new and existing customers month-over-month.**  
*What does this trend suggest about the balance between acquisition and retention efforts?*

### 5.1 Business Story & Context  
> Understanding how much revenue comes from **new** vs. **existing** customers each month reveals whether growth is driven more by acquisition or by repeat buying—and where to focus marketing and product efforts.

In [ ]:
# Annotating each transaction as New vs. Existing
sales_fp = sales.merge(first_purchase, on='customer_id')
sales_fp['customer_type'] = np.where(
    sales_fp['transaction_month'] == sales_fp['first_month'],
    'new',
    'existing'
)

# Aggregate monthly revenue by customer type
monthly_revenue = (
    sales_fp
      .groupby(['transaction_month', 'customer_type'])['revenue']
      .sum()
      .unstack(fill_value=0)
      .rename_axis(index='month')
)

# Compute total and % share
monthly_revenue['total'] = monthly_revenue['new'] + monthly_revenue['existing']
monthly_revenue['new_pct'] = (monthly_revenue['new'] / monthly_revenue['total'] * 100).round(1)
monthly_revenue['existing_pct'] = (monthly_revenue['existing'] / monthly_revenue['total'] * 100).round(1)

monthly_revenue.to_csv('../eda_outputs/revenue_new_existing.csv', index=False)

# Visualizations  

plt.figure(figsize=(10, 6))
monthly_revenue[['new', 'existing']].plot(kind='line', marker='o')
plt.title("Monthly Revenue: New vs. Existing Customers")
plt.xlabel("Month")
plt.ylabel("Revenue ($)")
plt.xticks(rotation=45)
plt.legend(title="Customer Type")
plt.tight_layout()
plt.savefig('../images/revenue_new_existing.png')
plt.show()

plt.figure(figsize=(10, 6))
ax = monthly_revenue[['new_pct', 'existing_pct']].plot(kind='line', figsize=(10, 6), marker='o', title="Monthly Revenue %: New vs. Existing Customers")
ax.set_xlabel("Month")
ax.set_ylabel("Revenue (%)")
plt.xticks(rotation=45)
ax.yaxis.set_major_formatter(PercentFormatter())
plt.legend(title="Customer Type")
plt.tight_layout()
plt.savefig('../images/revenue_new_existing_perc.png')
plt.show()

### 5.2 Results

Based on the **new vs. existing revenue mix**:

| Month    | New % | Existing % |
|----------|-------|------------|
| Jan – Mar | ≥ 84% | ≤ 16%     |
| Apr – Jun | 58–64% | 36–42%   |
| Jul – Sep | 40–48% | 52–60%   |
| Oct – Dec | 42–53% | 47–58%   |

## Question 6  
**Analyze the relationship between coupon usage and revenue generation.**  
*How can discount strategies be optimized to maximize revenue while maintaining profitability?*

### 6.1 Business Story & Context  
> Coupons can drive traffic and sales, but heavy discounting can erode margins.  
> By examining how different discount depths and coupon usage correlate with total and per-transaction revenue, we can identify the “sweet spot” for promotions that maximizes topline without sacrificing profitability.

In [ ]:
# Merge sales with discounts to get the discount_pct for each transaction
# discounts.month is a Period[M] matching sales.transaction_month
sales_disc = (
    sales
    .merge(
        discounts.assign(month=discounts['month'].astype('period[M]')),
        left_on=['transaction_month', 'product_category'],
        right_on=['month', 'product_category'],
        how='left'
    )
)

# Fill missing discount_pct with 0 (no coupon offered)
sales_disc['discount_pct'] = sales_disc['discount_pct'].fillna(0)

# Compute “effective” discount only when coupon was used
sales_disc['eff_discount_pct'] = np.where(
    sales_disc['coupon_status'] == 'Used',
    sales_disc['discount_pct'],
    0
)

# Aggregate by discount bucket
coupon_analysis = (
    sales_disc
    .groupby('eff_discount_pct')
    .agg(
        transactions=('transaction_id', 'count'),
        total_revenue=('revenue', 'sum'),
        avg_revenue=('revenue', 'mean')
    )
    .sort_index()
)

# Compute share of total revenue
coupon_analysis['rev_share_pct'] = (
    coupon_analysis['total_revenue'] 
    / coupon_analysis['total_revenue'].sum() 
    * 100
).round(1)

coupon_analysis.to_csv('../eda_outputs/coupon_vs_revenue.csv', index=False)

# Revenue share by discount depth
plt.figure(figsize=(8, 4))
plt.bar(
    coupon_analysis.index.astype(str),
    coupon_analysis['rev_share_pct']
)
plt.title('Share of Total Revenue by Discount %')
plt.xlabel('Effective Discount %')
plt.ylabel('Revenue Share (%)')
plt.tight_layout()
plt.savefig('../images/coupon_vs_revenue_perc.png')
plt.show()

# Avg transaction revenue by discount depth
plt.figure(figsize=(8, 4))
plt.bar(
    coupon_analysis.index.astype(str),
    coupon_analysis['avg_revenue']
)
plt.title('Avg Order Revenue by Discount %')
plt.xlabel('Effective Discount %')
plt.ylabel('Avg Revenue ($)')
plt.tight_layout()
plt.savefig('../images/discount_avg_revenue.png')
plt.show()

### 6.2 Results  

| Discount % | Transactions | Total Revenue ($) | Avg Revenue ($) | Rev. Share (%) |
|------------|--------------|-------------------|-----------------|----------------|
| 0%         | 35,146       | 3,118,390.44      | 88.73           | 66.8           |
| 10%        | 5,933        |   519,420.52      | 87.55           | 11.1           |
| 20%        | 6,000        |   510,646.92      | 85.11           | 10.9           |
| 30%        | 5,845        |   522,336.74      | 89.36           | 11.2           |

---

## Question 7  
**Identify the top-performing products and analyze the factors driving their success.**  
*How can this insight inform inventory management and promotional strategies?*

### 7.1 Business Story & Context  
> A small subset of SKUs often drives a large share of revenue and demand.  
> By pinpointing which products sell the most (and why—e.g., price point, coupon use, category), you can optimize stock levels, prioritize high-ROI promotions, and avoid overstock or stock-outs.

In [ ]:
# Reuse sales_disc from Q6 (sales merged with discounts + eff_discount_pct)
# If sales_disc is not in scope, recreate it:
sales_disc = (
    sales
    .merge(
        discounts.assign(month=discounts['month'].astype('period[M]')),
        left_on=['transaction_month', 'product_category'],
        right_on=['month','product_category'],
        how='left'
    )
    .assign(
        discount_pct=lambda df: df['discount_pct'].fillna(0),
        eff_discount_pct=lambda df: (df['discount_pct'].where(df['coupon_status']=='Used', 0))
    )
)

# Compute per-product metrics
product_metrics = (
    sales_disc
    .groupby(['product_sku','product_description','product_category'])
    .agg(
        total_revenue=('revenue','sum'),
        total_units_sold=('quantity','sum'),
        avg_price=('avg_price','mean'),
        coupon_rate=('coupon_status', lambda x: (x=='Used').mean()),
        avg_eff_discount=('eff_discount_pct','mean')
    )
    .sort_values('total_revenue', ascending=False)
)

# Select top 10 products by revenue
top_products = product_metrics.head(10).round({
    'total_revenue': 2,
    'avg_price': 2,
    'coupon_rate': 3,
    'avg_eff_discount': 1
})

top_products.to_csv('../eda_outputs/top_products.csv', index=False)

# Visualizations: Top 10 products by revenue
by_sku = top_products.copy()
by_sku.index = by_sku.index.get_level_values('product_sku')

plt.figure(figsize=(12, 6))
by_sku['total_revenue'].plot(
    kind='bar',
    legend=False
)
plt.title("Top 10 Products by Total Revenue")
plt.xlabel("Product SKU")
plt.ylabel("Total Revenue ($)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../images/top_products.png')
plt.show()

### Results : (Top 5 products)

| Product SKU       | Description                                               | Category | Total Rev ($) | Units Sold | Avg Price ($) | Coupon Rate | Avg Disc (%) |
|-------------------|-----------------------------------------------------------|----------|---------------|------------|---------------|-------------|--------------|
| GGOENEBJ079499    | Nest Learning Thermostat 3rd Gen-USA – Stainless Steel    | nest-usa |   688,916.34  | 4,570      | 150.98        | 34.3%       | 6.8%         |
| GGOENEBQ078999    | Nest Cam Outdoor Security Camera – USA                    | nest-usa |   629,977.12  | 5,206      | 121.81        | 32.8%       | 6.5%         |
| GGOENEBB078899    | Nest Cam Indoor Security Camera – USA                     | nest-usa |   528,612.93  | 4,402      | 120.21        | 32.0%       | 6.3%         |
| GGOENEBQ079099    | Nest Protect Smoke + CO White Battery Alarm-USA          | nest-usa |   213,819.16  | 2,683      | 79.84         | 33.8%       | 6.4%         |
| GGOENEBQ079199    | Nest Protect Smoke + CO White Wired Alarm-USA           | nest-usa |   212,495.57  | 2,670      | 79.75         | 37.0%       | 7.2%         |

---

## Question 8  
**Analyze the relationship between monthly marketing spend and revenue.**  
*Are there any months where marketing efforts yielded disproportionately high or low returns? How can marketing strategies be adjusted to improve ROI?*

### 8.1 Business Story & Context  
> Every rupee spent on marketing should generate more than a rupee in revenue.  
> By comparing monthly marketing spend (online + offline) against revenue, and computing an ROI metric, we can spot months of over- or under-performance and reallocate budget for maximum impact.

In [ ]:
# Aggregate monthly marketing spend
monthly_spend = (
    marketing
    .groupby('marketing_month')
    .agg(
        offline_spend=('offline_spend', 'sum'),
        online_spend=('online_spend', 'sum')
    )
    .assign(
        total_spend=lambda df: df['offline_spend'] + df['online_spend']
    )
)

# Combine with our monthly revenue
# monthly_revenue['total'] holds total revenue by transaction_month
promo_df = (
    monthly_spend
    .merge(
        monthly_revenue['total'].rename('revenue'),
        left_index=True,
        right_index=True,
        how='left'
    )
)

# Compute simple ROI = revenue / total_spend
promo_df['roi'] = (promo_df['revenue'] / promo_df['total_spend']).round(2)

promo_df.to_csv('../eda_outputs/marketing_roi.csv', index=False)

# Visualizations  

# Revenue vs. Marketing Spend scatter
plt.figure(figsize=(8, 6))
plt.scatter(
    promo_df['total_spend'],
    promo_df['revenue']
)
plt.title("Revenue vs. Total Marketing Spend by Month")
plt.xlabel("Total Marketing Spend ($)")
plt.ylabel("Revenue ($)")
plt.tight_layout()
plt.savefig('../images/marketing_roi.png')
plt.show()

# ROI by Month bar chart
plt.figure(figsize=(10, 5))
promo_df['roi'].plot(kind='bar', legend=False)
plt.title("Monthly Marketing ROI (Revenue ÷ Spend)")
plt.xlabel("Month")
plt.ylabel("ROI")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../images/marketing_roi_monthly.png')
plt.show()

### 8.2 Results  

| Month    | Offline Spend ($) | Online Spend ($) | Total Spend ($) | Revenue ($)  | ROI  |
|----------|-------------------|------------------|-----------------|--------------|------|
| 2019-01  | 96,600            | 58,328.95        | 154,928.95      | 403,624.58   | 2.61 |
| 2019-02  | 81,300            | 55,807.92        | 137,107.92      | 310,819.80   | 2.27 |
| 2019-03  | 73,500            | 48,750.09        | 122,250.09      | 349,608.09   | 2.86 |
| 2019-04  | 96,000            | 61,026.83        | 157,026.83      | 401,618.42   | 2.56 |
| 2019-05  | 65,500            | 52,759.64        | 118,259.64      | 307,763.42   | 2.60 |
| 2019-06  | 80,500            | 53,818.14        | 134,318.14      | 321,081.38   | 2.39 |
| 2019-07  | 67,500            | 52,717.85        | 120,217.85      | 372,638.07   | 3.10 |
| 2019-08  | 85,500            | 57,404.15        | 142,904.15      | 401,210.37   | 2.81 |
| 2019-09  | 83,000            | 52,514.54        | 135,514.54      | 360,548.40   | 2.66 |
| 2019-10  | 93,500            | 57,724.65        | 151,224.65      | 409,681.28   | 2.71 |
| 2019-11  | 93,000            | 68,144.96        | 161,144.96      | 508,942.62   | 3.16 |
| 2019-12  | 122,000           | 76,648.75        | 198,648.75      | 523,258.19   | 2.63 |

---

## Question 9  
**Evaluate the effectiveness of marketing campaigns by comparing marketing spend to revenue generated.**  
*Are there opportunities to reallocate resources for better results?*

### 9.1 Business Story & Context  
> Assessing which channels deliver the best return allows us to shift budgets away from under-performing tactics into high-ROI campaigns—maximizing impact per rupee spent.

In [ ]:
# Calculate ROI by channel
channel_roi = promo_df.copy().assign(
    roi_offline=lambda df: (df['revenue'] / df['offline_spend']).round(2),
    roi_online =lambda df: (df['revenue'] / df['online_spend']).round(2)
)[[
    'offline_spend','online_spend','revenue','roi_offline','roi_online'
]]

channel_roi.to_csv('../eda_outputs/channel_roi.csv', index=False)

# Visualizations  
# Side-by-side ROI comparison
ax = channel_roi[['roi_offline','roi_online']].plot(
    kind='bar', figsize=(10, 6), rot=45
)
ax.set_title("Monthly ROI: Offline vs. Online Marketing")
ax.set_xlabel("Month")
ax.set_ylabel("ROI (Revenue ÷ Spend)")
plt.tight_layout()
plt.savefig('../images/channel_roi.png')
plt.show()

### 9.2. Results  

| Month    | Offline Spend ($) | Online Spend ($) | Total Spend ($) | Revenue ($)  | Offline ROI | Online ROI |
|----------|-------------------|------------------|-----------------|--------------|-------------|------------|
| 2019-01  | 96,600            | 58,328.95        | 154,928.95      | 403,624.58   | 4.18×       | 6.92×      |
| 2019-02  | 81,300            | 55,807.92        | 137,107.92      | 310,819.80   | 3.82×       | 5.57×      |
| 2019-03  | 73,500            | 48,750.09        | 122,250.09      | 349,608.09   | 4.76×       | 7.17×      |
| 2019-04  | 96,000            | 61,026.83        | 157,026.83      | 401,618.42   | 4.18×       | 6.58×      |
| 2019-05  | 65,500            | 52,759.64        | 118,259.64      | 307,763.42   | 4.70×       | 5.83×      |
| 2019-06  | 80,500            | 53,818.14        | 134,318.14      | 321,081.38   | 3.99×       | 5.97×      |
| 2019-07  | 67,500            | 52,717.85        | 120,217.85      | 372,638.07   | 5.52×       | 7.07×      |
| 2019-08  | 85,500            | 57,404.15        | 142,904.15      | 401,210.37   | 4.69×       | 6.99×      |
| 2019-09  | 83,000            | 52,514.54        | 135,514.54      | 360,548.40   | 4.34×       | 6.87×      |
| 2019-10  | 93,500            | 57,724.65        | 151,224.65      | 409,681.28   | 4.38×       | 7.10×      |
| 2019-11  | 93,000            | 68,144.96        | 161,144.96      | 508,942.62   | 5.47×       | 7.47×      |
| 2019-12  | 122,000           | 76,648.75        | 198,648.75      | 523,258.19   | 4.29×       | 6.83×      |

---

## Question 10  
**Segment customers into groups such as Premium, Gold, Silver, and Standard using RFM techniques.**  
*What targeted strategies can be developed for each segment to improve retention and revenue?*

### 10.1 Business Story & Context  
> Not all customers are alike—some buy often and spend big, others rarely return.  
> **RFM segmentation** (Recency, Frequency, Monetary) lets us tailor retention and upsell strategies to each group, maximizing ROI on marketing and loyalty investments.

In [ ]:
# Compute RFM metrics
# Use the latest transaction date as reference
last_date = sales['transaction_date'].max()

# Build a table of customer-level RFM
rfm = sales.groupby('customer_id').agg(
    recency_days=('transaction_date', lambda x: (last_date - x.max()).days),
    frequency=('transaction_id', 'nunique'),
    monetary=('revenue', 'sum')
).reset_index()

# Define quartiles for scoring
quantiles = rfm[['recency_days','frequency','monetary']].quantile([0.25,0.50,0.75]).to_dict()

# Score each metric on a 1–4 scale
def r_score(x):
    if x <= quantiles['recency_days'][0.25]: return 4
    if x <= quantiles['recency_days'][0.50]: return 3
    if x <= quantiles['recency_days'][0.75]: return 2
    return 1

def fm_score(x, col):
    if x <= quantiles[col][0.25]: return 1
    if x <= quantiles[col][0.50]: return 2
    if x <= quantiles[col][0.75]: return 3
    return 4

rfm['R_score'] = rfm['recency_days'].apply(r_score)
rfm['F_score'] = rfm['frequency'].apply(lambda x: fm_score(x, 'frequency'))
rfm['M_score'] = rfm['monetary'].apply(lambda x: fm_score(x, 'monetary'))

# Combine into segments
# Premium if all scores ≥3, Gold if all ≥2, Silver if sum ≥5, else Standard
rfm['segment'] = 'Standard'
rfm.loc[(rfm[['R_score','F_score','M_score']] >= 3).all(axis=1), 'segment'] = 'Premium'
rfm.loc[
    (rfm[['R_score','F_score','M_score']] >= 2).all(axis=1) & 
    (rfm['segment'] != 'Premium'),
    'segment'
] = 'Gold'
rfm.loc[
    (rfm['segment']=='Standard') & (rfm[['R_score','F_score','M_score']].sum(axis=1) >= 5),
    'segment'
] = 'Silver'

# Inspect segment distribution
segment_counts = rfm['segment'].value_counts().rename_axis('segment').reset_index(name='count')

segment_counts.to_csv('../eda_outputs/segment_counts.csv', index=False)


#Visualizations 

plt.figure(figsize=(6,4))
plt.bar(segment_counts['segment'], segment_counts['count'])
plt.title("Customer Count by RFM Segment")
plt.xlabel("Segment")
plt.ylabel("Number of Customers")
plt.tight_layout()
plt.savefig('../images/segment_counts.png')
plt.show()

### 10.2 Results  

**Sample RFM table (first 5 customers):**

| customer_id | recency_days | frequency | monetary   | R_score | F_score | M_score | segment  |
|-------------|--------------|-----------|------------|---------|---------|---------|----------|
| 12346       | 107          | 1         | $30.99     | 3       | 1       | 1       | Silver   |
| 12347       | 59           | 31        | $13,834.90 | 3       | 4       | 4       | Premium  |
| 12348       | 73           | 8         | $1,442.12  | 3       | 2       | 2       | Gold     |
| 12350       | 17           | 11        | $1,360.07  | 4       | 2       | 2       | Gold     |
| 12356       | 107          | 13        | $1,442.47  | 3       | 3       | 2       | Gold     |

**Segment distribution:**

| Segment   | Count |
|-----------|-------|
| Premium   | 417   |
| Silver    | 406   |
| Gold      | 384   |
| Standard  | 261   |

---

## Question 11  
**Analyze the revenue contribution of each customer segment.**  
*How can the company focus its efforts on high-value segments while nurturing lower-value segments?*

### 11.1 Business Story & Context  
> Understanding which RFM segments drive the most revenue informs where to double down with premium service and where to invest in growth programs.  
> We’ll quantify each segment’s share of total revenue and average spend per customer to guide resource allocation.

In [ ]:
# Aggregate revenue by segment
segment_rev = (
    rfm
    .groupby('segment')['monetary']
    .agg(
        total_revenue=('sum'),
        avg_revenue=('mean'),
        customer_count=('count')
    )
    .assign(
        rev_share_pct=lambda df: (df['total_revenue'] / df['total_revenue'].sum() * 100).round(1)
    )
    .sort_values('total_revenue', ascending=False)
)

segment_rev.to_csv('../eda_outputs/segment_rev.csv', index=False)

# Visualizations  
# Revenue share by segment
plt.figure(figsize=(6,4))
plt.bar(segment_rev.index, segment_rev['rev_share_pct'])
plt.title("Revenue Share by Customer Segment")
plt.xlabel("Segment")
plt.ylabel("Revenue Share (%)")
plt.tight_layout()
plt.savefig('../images/segment_rev_cust.png')
plt.show()

# Avg revenue per customer by segment
plt.figure(figsize=(6,4))
plt.bar(segment_rev.index, segment_rev['avg_revenue'])
plt.title("Average Revenue per Customer by Segment")
plt.xlabel("Segment")
plt.ylabel("Avg Spend ($)")
plt.tight_layout()
plt.savefig('../images/avg_segment_rev.png')
plt.show()

### 11.2 Results  

| Segment   | Customer Count | Total Revenue ($) | % of Total Rev | Avg Revenue per Cust ($) |
|-----------|----------------|-------------------|----------------|--------------------------|
| **Premium**   | 417            | 2,868,407.74      | 61.4 %         | 6,878.68                 |
| **Gold**      | 384            |   905,958.68      | 19.4 %         | 2,359.27                 |
| **Silver**    | 406            |   806,899.87      | 17.3 %         | 1,987.44                 |
| **Standard**  | 261            |    89,528.33      |  1.9 %         |   343.02                 |

---

## Question 12  
**Group customers by their month of first purchase and analyze retention rates over time.**  
*Which cohorts exhibit the highest and lowest retention rates? What strategies can be implemented to improve retention for weaker cohorts?*

### 12.1 Business Story & Context  
> By tracking each “first-purchase month” cohort’s repeat behavior over subsequent months, we can see which acquisition periods produce the stickiest customers—and where to shore up our lifecycle programs to reduce churn.

In [ ]:
# unique customer counts by cohort and transaction month
cohort = (
    sales
    .merge(first_purchase, on='customer_id')
    .groupby(['first_month','transaction_month'])['customer_id']
    .nunique()
    .reset_index(name='n_customers')
)

# Compute period number: months since first purchase
cohort['period_number'] = (cohort['transaction_month'] - cohort['first_month']).apply(lambda x: x.n)

# Pivot into a retention matrix (including period 0 = cohort size)
cohort_pivot = cohort.pivot(
    index='first_month',
    columns='period_number',
    values='n_customers'
).fillna(0)

# Compute retention rates by dividing by cohort size (period 0)
retention_table = (
    cohort_pivot
    .div(cohort_pivot[0], axis=0)
    .drop(columns=0)
    .round(2)
)

retention_table.to_csv('../eda_outputs/retention_rates.csv', index=False)

# Visualizations  
plt.figure(figsize=(15, 6))
sns.heatmap(retention_table, annot=True, fmt='.0%', cmap='YlGnBu')
plt.title('Cohort Retention Matrix')
plt.xlabel('Months Since Acquisition')
plt.ylabel('Acquisition Month')
plt.tight_layout()
plt.savefig('../images/cohort_retention.png')
plt.show()

### 12.2 Results  

| Cohort   | 1 mo | 2 mo | 3 mo | 4 mo | 5 mo | 6 mo |
|----------|------|------|------|------|------|------|
| **2019-01** |  6 % | 11 % | 16 % | 11 % | 20 % | 16 % |
| **2019-02** |  7 % |  9 % | 17 % | 18 % | 23 % | 20 % |
| **2019-03** | 10 % | 20 % | 14 % | 18 % | 19 % | 12 % |
| **2019-04** |  9 % | 15 % | 15 % | 11 % |  9 % |  6 % |
| **2019-05** | 11 % |  8 % | 12 % |  9 % | 12 % | 12 % |
| **2019-06** | 15 % | 16 % |  9 % |  8 % | 10 % |  8 % |

---

## Question 13  
**Analyze the lifetime value of customers acquired in different months.**  
*How can this insight inform acquisition and retention strategies?*


### 13.1 Business Story & Context  
> Lifetime Value (LTV) measures the total revenue a customer generates over their entire relationship.  
> Comparing cohort LTV by acquisition month reveals which marketing windows yield the most valuable customers—and where to fine-tune spend and nurture tactics.

In [ ]:
# Compute total revenue per customer (their lifetime value)
ltv_per_customer = (
    sales
    .groupby('customer_id')['revenue']
    .sum()
    .rename('ltv')
    .reset_index()
)

# Merge LTV back to their cohort month
ltv_cohort = (
    first_purchase
    .merge(ltv_per_customer, on='customer_id')
    .groupby('first_month')
    .agg(
        cohort_size=('customer_id', 'nunique'),
        total_ltv=('ltv', 'sum'),
        avg_ltv=('ltv', 'mean')
    )
    .round(2)
    .reset_index()
)

ltv_cohort.to_csv('../eda_outputs/ltv_cohort.csv', index=False)


# Visualisation

plt.figure(figsize=(10, 5))
plt.plot(ltv_cohort['first_month'].astype(str), ltv_cohort['avg_ltv'], marker='o')
plt.title("Average Customer LTV by Acquisition Month")
plt.xlabel("Acquisition Month")
plt.ylabel("Avg LTV ($)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../images/ltv_cohort.png')
plt.show()

### 13.2 Results  

| Cohort   | Cohort Size | Total LTV ($) | Avg LTV per Customer ($) |
|----------|-------------|---------------|---------------------------|
| **2019-01** | 215         | 1,037,320.06  | 4,824.74                  |
| **2019-02** | 96          |   540,338.52  | 5,628.53                  |
| **2019-03** | 177         |   668,895.39  | 3,779.07                  |
| **2019-04** | 163         |   449,331.26  | 2,756.63                  |
| **2019-05** | 112         |   332,698.60  | 2,970.52                  |
| **2019-06** | 137         |   292,800.81  | 2,137.23                  |
| **2019-07** | 94          |   240,255.54  | 2,555.91                  |
| **2019-08** | 135         |   259,011.87  | 1,918.61                  |
| **2019-09** | 78          |   151,664.24  | 1,944.41                  |
| **2019-10** | 87          |   229,976.73  | 2,643.41                  |
| **2019-11** | 68          |   221,691.63  | 3,260.17                  |
| **2019-12** | 106         |   246,809.97  | 2,328.40                  |

---

## Question 14  
**Do customers who use coupons have a different average transaction value compared to those who do not?**  
*Conduct a statistical test to validate this hypothesis. What implications does this have for the company’s discount and coupon strategies?*

### 14.1 Business Story & Context  
> Coupons are a powerful lever to drive sales volume, but they also carry margin risk if coupon-users consistently spend less per order.  
> By comparing average transaction values for “coupon used” vs. “no coupon” groups and running a statistical significance test, we can determine whether discount‐driven orders truly differ in size—and adjust our coupon strategy (depth, targeting, eligibility) accordingly.  

In [ ]:
#  Prepare the two groups
# “Used” vs. “Not Used” based on coupon_status
used = sales_disc.loc[sales_disc['coupon_status']=='Used', 'total_amount']
not_used = sales_disc.loc[sales_disc['coupon_status']!='Used', 'total_amount']

# Calculate group stats
mean_used     = used.mean()
mean_not_used = not_used.mean()
n_used        = used.size
n_not_used    = not_used.size

print(f"Used coupon:      n = {n_used}, mean = {mean_used:.2f}")
print(f"No coupon used:   n = {n_not_used}, mean = {mean_not_used:.2f}")

# Independent t-test (unequal variances)
tstat, pval = ttest_ind(used, not_used, equal_var=False)
print(f"t-statistic = {tstat:.3f}, p-value = {pval:.3e}")

### 14.2 Results  

| Group            | N      | Avg Transaction Value ($) |
|------------------|--------|---------------------------|
| **Used Coupon**      | 17,904 | 97.65                     |
| **No Coupon Used**   | 35,020 | 99.35                     |

- **t‐statistic:** -1.084  
- **p‐value:** 0.2785  

> With a p‐value > 0.05, the difference in average transaction value between coupon users and non‐users is **not** statistically significant.

---

## Question 15  
**Do purchase behaviors (e.g., order frequency, order value) vary significantly across different demographic groups or pricing factors (e.g., delivery charges)?**  
*Test for differences in purchase behavior across locations, tenure groups, and delivery‐charge tiers. How can these insights inform personalized marketing and pricing strategies?*

### 15.1 Business Story & Context  
> Customer value drivers—how often they buy and how much they spend—can differ by where they live, how long they’ve been with us, and even by delivery fees.  
> By uncovering statistically significant differences, we can tailor promotions, pricing, and communication to each segment for maximum impact.


In [ ]:
# Join customer demographics
sales_demo = sales.merge(customers, on='customer_id')

# Define tenure groups (in months) as proxy for “age”
sales_demo['tenure_group'] = pd.cut(
    sales_demo['tenure_months'],
    bins=[0, 12, 36, np.inf],
    labels=['<1yr','1-3yr','>3yr']
)

# Define delivery‐charge tiers
sales_demo['delivery_tier'] = pd.cut(
    sales_demo['delivery_charges'],
    bins=[-np.inf, 5, 10, np.inf],
    labels=['Low (≤5)','Mid (5-10)','High (>10)']
)

# Compute per-customer metrics
cust_metrics = (
    sales_demo
    .groupby('customer_id')
    .agg(
        frequency=('transaction_id','nunique'),
        avg_value =('total_amount','mean'),
        location    =('location','first'),
        tenure_group=('tenure_group','first'),
        delivery_tier=('delivery_tier','first')
    )
    .reset_index()
)

# Summarize by group
loc_summary = cust_metrics.groupby('location')[['frequency','avg_value']].describe().round(2)
tenure_summary = cust_metrics.groupby('tenure_group', observed=False)[['frequency','avg_value']].describe().round(2)
delivery_summary = cust_metrics.groupby('delivery_tier', observed=False)[['frequency','avg_value']].describe().round(2)

print("Location Summary:\n", loc_summary)
print("\nTenure Summary:\n", tenure_summary)
print("\nDelivery-Tier Summary:\n", delivery_summary)

# ANOVA tests
f_loc, p_loc = f_oneway(*[g['avg_value'].values for _,g in cust_metrics.groupby('location')])
f_tenure, p_tenure = f_oneway(*[g['avg_value'].values for _,g in cust_metrics.groupby('tenure_group', observed=False)])
f_deliv, p_deliv = f_oneway(*[g['avg_value'].values for _,g in cust_metrics.groupby('delivery_tier', observed=False)])

print(f"\nANOVA Avg Value by Location: F={f_loc:.2f}, p={p_loc:.3f}")
print(f"ANOVA Avg Value by Tenure:   F={f_tenure:.2f}, p={p_tenure:.3f}")
print(f"ANOVA Avg Value by Delivery: F={f_deliv:.2f}, p={p_deliv:.3f}")

### 15.2 Results  

#### A. Mean Purchase Metrics by Group

**By Location**  
| Location       | Mean Frequency | Mean Avg Transaction Value ($) |
|----------------|---------------:|-------------------------------:|
| California     |          17.37 |                          95.93 |
| Chicago        |          20.36 |                          99.04 |
| New Jersey     |          15.36 |                          98.63 |
| New York       |          17.24 |                          94.28 |
| Washington DC  |          18.89 |                         112.29 |

**By Tenure Group**  
| Tenure Group | Mean Frequency | Mean Avg Transaction Value ($) |
|--------------|---------------:|-------------------------------:|
| <1yr         |          17.65 |                          98.33 |
| 1–3yr        |          18.63 |                          95.69 |
| >3yr         |          17.67 |                         100.53 |

**By Delivery-Charge Tier**  
| Delivery Tier | Mean Frequency | Mean Avg Transaction Value ($) |
|---------------|---------------:|-------------------------------:|
| Low (≤5)      |          31.00 |                         106.51 |
| Mid (5–10)    |          17.99 |                          95.13 |
| High (>10)    |          18.83 |                         110.00 |

#### B. ANOVA Test Results (Avg Transaction Value)

- **Location:** F = 1.59, p = 0.175 (no significant difference)  
- **Tenure Group:** F = 0.91, p = 0.402 (no significant difference)  
- **Delivery-Charge Tier:** F = 6.56, p = 0.001 (significant difference)

---

## Question 16  
**Does customer tenure impact purchase frequency?**  
*Analyze the relationship between customer tenure and purchase frequency. How can this insight inform loyalty and engagement strategies?*

### 16.1 Business Story & Context  
> Customers who have been with us longer may purchase more (or less) often than newer customers.  
> Understanding this relationship helps tailor loyalty programs, re-engagement campaigns, and onboarding tactics to each tenure cohort.

In [ ]:
# Summarize purchase frequency by tenure group
freq_summary = cust_metrics.groupby('tenure_group', observed=False)['frequency'].describe().round(2)
print("Frequency by Tenure Group:\n", freq_summary)

# ANOVA test to see if mean frequency differs across tenure groups
f_freq, p_freq = f_oneway(
    *[grp['frequency'].values for _, grp in cust_metrics.groupby('tenure_group', observed=False)]
)
print(f"\nANOVA Frequency by Tenure: F={f_freq:.2f}, p={p_freq:.3f}")

freq_summary.to_csv('../eda_outputs/freq_summary.csv', index=False)

### 16.2 Results  

**Purchase Frequency by Tenure Group**  
| Tenure Group | Count | Mean Frequency | Std Dev | Min | 25% | 50% | 75% | Max  |
|--------------|------:|---------------:|--------:|----:|----:|----:|----:|-----:|
| <1yr         | 327   |          17.65 |   20.37 |  1.0|  5.0| 11.0| 24.5| 177.0|
| 1–3yr        | 727   |          18.63 |   28.08 |  1.0|  5.0| 11.0| 23.5| 328.0|
| >3yr         | 414   |          17.67 |   22.40 |  1.0|  5.0| 11.0| 23.0| 291.0|

**ANOVA Test**  
- F-statistic = 0.27  
- p-value     = 0.760  

> p-value > 0.05 ⇒ no statistically significant difference in purchase frequency across tenure cohorts.

---

## Question 17  
**Analyze the relationship between delivery charges and order behavior.**  
*Are there opportunities to optimize delivery pricing to increase order quantities or revenue?*

### 17.1 Business Story & Context  
> Delivery fees can affect how much customers add to their cart (order value) and whether they complete an order at all (quantity ordered).  
> By examining order behavior across low, mid, and high delivery‐charge tiers, and testing for significant differences, we can uncover pricing levers (e.g., free‐shipping thresholds or tiered fees) to boost basket size and overall revenue.

In [ ]:
# Define delivery-charge tiers on the transaction level
sales['delivery_tier'] = pd.cut(
    sales['delivery_charges'],
    bins=[-np.inf, 5, 10, np.inf],
    labels=['Low (≤$5)', 'Mid ($5-10)', 'High (>$10)']
)

# Compute order-level metrics by tier
order_metrics = (
    sales
    .groupby('delivery_tier', observed=False)
    .agg(
        order_count     = ('transaction_id', 'count'),
        avg_quantity    = ('quantity',       'mean'),
        avg_order_value = ('total_amount',   'mean'),
        total_revenue   = ('revenue',        'sum')
    )
    .round(2)
)

# ANOVA to test differences in avg_order_value across tiers
groups = [grp['total_amount'].values for _, grp in sales.groupby('delivery_tier', observed=False)]
f_val, p_val = f_oneway(*groups)

print(order_metrics)
print(f"\nANOVA Avg Order Value by Delivery Tier: F = {f_val:.2f}, p = {p_val:.3f}")

order_metrics.to_csv('../eda_outputs/order_metrics.csv', index=False)

### 17.2 Results  

| Delivery Tier | Order Count | Avg Quantity | Avg Order Value ($) | Total Revenue ($)   |
|---------------|------------:|-------------:|--------------------:|---------------------:|
| Low (≤$5)     |         162 |          1.31|               120.66|              19,547.24|
| Mid ($5–10)   |      43,017 |          3.13|                93.71|           3,764,379.19|
| High (>$10)   |       9,745 |         10.57|               120.73|             886,868.19|

**ANOVA on Avg Order Value by Delivery Tier**  
- F-statistic = 106.52  
- p-value     = 0.000 (significant at p<0.001)

---

## Question 18  
**Evaluate how taxes and delivery charges influence customer spending behavior.**  
*Are there opportunities to adjust pricing strategies to improve customer satisfaction and revenue?*

### 18.1 Business Story & Context  
> Both product-level GST rates and delivery fees add to the final price customers pay.  
> Understanding how different tax brackets and delivery-charge tiers affect basket size and order value helps optimize pricing (thresholds, bundling, and promotions) to balance top-line revenue and customer satisfaction."

In [ ]:
# Merge sales with GST rates
sales_tax = sales.merge(tax, on='product_category')

# Compute tax amount and total spend including tax
sales_tax['tax_amt'] = sales_tax['revenue'] * sales_tax['gst']
sales_tax['spend_inc_tax'] = sales_tax['total_amount'] + sales_tax['tax_amt']

# Summarize spend_inc_tax by GST rate
gst_summary = (
    sales_tax
    .groupby('gst')['spend_inc_tax']
    .agg(['count','mean','std'])
    .round(2)
)

# ANOVA test across GST groups
f_tax, p_tax = f_oneway(
    *[grp['spend_inc_tax'].values for _, grp in sales_tax.groupby('gst')]
)

# Correlation between delivery fee and spend_inc_tax
corr = sales_tax[['delivery_charges','spend_inc_tax']].corr().iloc[0,1]

print("GST Summary:\n", gst_summary)
print(f"\nANOVA by GST: F = {f_tax:.2f}, p = {p_tax:.3f}")
print(f"\nCorrelation between delivery charges and spend_inc_tax: {corr:.3f}")

order_metrics.to_csv('../eda_outputs/order_metrics.csv', index=False)

### 18.2 Results  

**A. Spend Including Tax by GST Rate**  
| GST Rate | Count | Mean Spend ($) | Std Dev ($) |
|---------:|------:|---------------:|------------:|
| 0.05     |  4,145|         188.98 |      372.92 |
| 0.10     | 21,314|         160.88 |      151.24 |
| 0.12     |    122|          59.43 |       97.61 |
| 0.18     | 27,343|          55.68 |      140.19 |

- **ANOVA by GST:** F = 1782.59, p < 0.001 (significant differences in spend among GST brackets)  
- **Correlation (delivery fee vs. spend incl. tax):** r = 0.176 (weak positive relationship)

---

## Question 19  
**Identify seasonal trends in sales by category and location.**  
*How can the company prepare for peak and off‐peak seasons to maximize revenue?*

### 19.1 Business Story & Context  
> Different product categories and regions often experience their own “seasons” of high and low demand.  
> By mapping monthly revenue patterns by category and by location, we can anticipate inventory needs, staffing, and marketing budgets—ensuring we’re fully stocked and promoted when demand peaks, and optimizing costs in slower months.

In [ ]:
# Join sales with customer location
sales_loc = sales.merge(customers[['customer_id','location']], on='customer_id')

# Aggregate monthly revenue by category
category_season = (
    sales_loc
    .groupby(['transaction_month','product_category'])['revenue']
    .sum()
    .unstack(fill_value=0)
)

# Aggregate monthly revenue by location
location_season = (
    sales_loc
    .groupby(['transaction_month','location'])['revenue']
    .sum()
    .unstack(fill_value=0)
)

# Display the first few rows of each
print("Category Seasonality (first 6 months):\n", category_season.head(6))
print("\nLocation Seasonality (first 6 months):\n", location_season.head(6))

category_season.to_csv('../eda_outputs/category_season.csv', index=False)
location_season.to_csv('../eda_outputs/location_season.csv', index=False)

In [ ]:
# Visualisations

# Plot each category’s monthly revenue
plt.figure(figsize=(10, 6))
for cat in category_season.columns:
    plt.plot(category_season.index.astype(str), category_season[cat], marker='o', label=cat)
plt.title("Monthly Revenue by Product Category")
plt.xlabel("Month")
plt.ylabel("Revenue ($)")
plt.xticks(rotation=45)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.tight_layout()
plt.savefig('../images/category_season.png')
plt.show()

# Plot each location’s monthly revenue
plt.figure(figsize=(10, 6))
for loc in location_season.columns:
    plt.plot(location_season.index.astype(str), location_season[loc], marker='o', label=loc)
plt.title("Monthly Revenue by Location")
plt.xlabel("Month")
plt.ylabel("Revenue ($)")
plt.xticks(rotation=45)
plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.tight_layout()
plt.savefig('../images/location_season.png')
plt.show()

### 19.2 Results  

#### A. Category Seasonality (First 6 Months)

| Month    | Apparel   | Bags      | Drinkware | Nest-USA   | Office   |
|----------|----------:|----------:|----------:|-----------:|---------:|
| 2019-01  |  38,300.87|  10,903.63|  14,599.09|   284,362.08|  27,309.50|
| 2019-02  |  37,990.09|  17,969.55|  14,334.22|   196,182.19|  20,994.91|
| 2019-03  |  59,059.02|  14,045.65|  21,228.20|   199,700.00|  27,783.12|
| 2019-04  |  88,138.14|  11,385.03|  27,090.43|   182,193.00|  36,507.38|
| 2019-05  |  58,850.90|   9,962.99|  16,424.25|   173,534.00|  23,022.42|
| 2019-06  |  45,941.01|  12,867.37|  17,622.04|   195,413.00|  22,718.12|

#### B. Location Seasonality (First 6 Months)

| Month    | California | Chicago  | New Jersey | New York  | Washington DC |
|----------|-----------:|---------:|-----------:|----------:|--------------:|
| 2019-01  | 150,241.63 |109,911.74|   31,475.72| 78,217.28 |      33,778.21|
| 2019-02  |  89,727.46 |108,031.09|   30,266.50| 44,748.86 |      38,045.89|
| 2019-03  |  99,243.75 |118,365.41|   19,393.74| 97,026.06 |      15,579.13|
| 2019-04  | 121,139.09 |152,811.80|   44,989.45| 63,181.57 |      19,496.51|
| 2019-05  |  86,602.49 |109,277.26|   28,295.03| 77,345.24 |       6,243.40|
| 2019-06  | 100,427.83 |100,742.75|   40,006.89| 68,613.44 |      11,290.47|

---

## Question 20  
**Analyze daily sales trends to identify high-performing and low-performing days.**  
*What strategies can be implemented to boost sales on slower days?*

### 20.1 Business Story & Context  
> Identifying which days of the week (and specific dates) generate the most and least revenue enables smarter staffing, inventory planning, and timing of promotional campaigns.  

In [ ]:
# Group by weekday
sales['weekday'] = sales['transaction_date'].dt.day_name()
weekday_summary = (
    sales
    .groupby('weekday')['revenue']
    .agg(count='count', total='sum', avg='mean')
    .reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
    .round(2)
)
print("Weekday Summary:\n", weekday_summary)

# daily time series
daily_ts = sales.groupby('transaction_date')['revenue'].sum().reset_index()
print("Daily Revenue Time Series (first 5 rows):\n", daily_ts.head())

weekday_summary.to_csv('../eda_outputs/weekday_summary.csv', index=False)
daily_ts.to_csv('../eda_outputs/daily_ts.csv', index=False)

In [ ]:
# Visualizations
# Bar chart: Avg revenue by weekday
plt.figure(figsize=(8,5))
weekday_summary['avg'].plot(kind='bar')
plt.title("Average Revenue by Weekday")
plt.xlabel("Weekday")
plt.ylabel("Avg Revenue ($)")
plt.tight_layout()
plt.savefig('../images/weekday_summary.png')
plt.show()

# 4.2 Line chart: Daily revenue over time
plt.figure(figsize=(12,5))
plt.plot(daily_ts['transaction_date'], daily_ts['revenue'])
plt.title("Daily Revenue Over Time")
plt.xlabel("Date")
plt.ylabel("Revenue ($)")
plt.tight_layout()
plt.savefig('../images/daily_ts.png')
plt.show()

### 20.2 – Results  
**Weekday Summary**  
| Weekday   | # Transactions | Total Revenue ($) | Avg. Revenue per Day ($) |
|-----------|----------------|-------------------|--------------------------:|
| Monday    | 4 464          | 365 626.90        | 81.91                     |
| Tuesday   | 4 611          | 396 819.65        | 86.06                     |
| Wednesday | 8 887          | 826 622.00        | 93.01                     |
| Thursday  | 9 000          | 840 433.85        | 93.38                     |
| Friday    | 9 266          | 872 004.86        | 94.11                     |
| Saturday  | 8 177          | 673 068.03        | 82.31                     |
| Sunday    | 8 519          | 696 219.33        | 81.73                     |

- **Highest average daily revenue** occurs on **Friday ($94.11 K)** and **Thursday ($93.38 K)**.  
- **Lowest average** on **Sunday ($81.73 K)** and **Monday ($81.91 K)**.  

**Daily Time Series**  
- The line chart shows sharp spikes on certain dates (likely major promotions or product launches) interspersed with a clear “sawtooth” weekly pattern.  
- Revenue gradually trends upward toward year-end with pronounced peaks.

---